# LIST 5

In [741]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import geopandas as gpd

from urllib import request
from geotext import GeoText

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [739]:
url = "https://www.gutenberg.org/files/103/103-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')

In [719]:
places = GeoText(raw)
cities = list(places.cities)

In [721]:
import csv

In [722]:
file = open('cities.csv',encoding="utf8")

In [723]:
csvreader = csv.reader(file)

In [724]:
all_cities = []
count = 1
for row in csvreader:
    if count == 1:
        count += 1
    else:
        rows = row[0].split(",")
        if rows[-2].strip('"') != '':
            population = float(rows[-2].strip('"'))
            if population >= 300000:
                all_cities.append(rows[1].strip('"'))

In [726]:
visited = []
for city in cities:
    if city in all_cities:
        visited.append(city)

In [728]:
geolocator = Nominatim(user_agent="http")

In [729]:
coor = []
for city in visited:
    try:
        location = geolocator.geocode(city)
        if location:
            coor.append([location.latitude, location.longitude])
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input")

In [730]:
def insert_city(path, city):
    if (city[1] >= path[0][1]) and  (abs(city[0]-path[-1][0])<50) :
        for v in range(0, len(path)):
            if city[1] < path[v][1]:
                path.insert((v), city)
                break
        if city not in path:
            path.append(city)
    return path 

In [731]:
def travel(visited):
    path_east = [[51.5073219, -0.1276474]]
    path_west = [[40, -180]]
    for i in range(1, len(coor)):
        if coor[i] in (path_east + path_west):
            pass
        else:
            if -180 <= coor[i][1] <= -30:
                path_west = insert_city(path_west, coor[i])
            else:
                path_east = insert_city(path_east, coor[i])

            
    path_west.append([51.5073219, -0.1276474])
    path_west.remove(path_west[0])
    return path_east + path_west

In [732]:
trv = travel(visited)
travel_map = []

for t in trv:
    location = geolocator.reverse(str(t[0])+","+str(t[1]))
    travel_map.append(location)

In [733]:
df = pd.DataFrame(travel_map, columns=['City Name', 'Coordinates'])
df.head(7)

,City Name,Coordinates
0,"Charles I, Charing Cross, St. James's, Covent ...","(51.5073171, -0.12765535893207325)"
1,"3, Rue Casimir Périer, Quartier des Invalides,...","(48.8588785, 2.3201019)"
2,Evangelisch-lutherische Hauptkirche St. Jacobi...,"(53.5503571, 10.000741333632924)"
3,"Ομόνοια, Πλατεία Ομονοίας, Ομόνοια, 1η Κοινότη...","(37.9840538, 23.727983)"
4,"1, حارة إمام, السويس, محافظة السويس, 43515, مصر","(29.9744948, 32.5369882)"
5,"بئر رباك, مديرية البريقة, محافظة عدن, اليمن","(12.834043695216634, 44.91802183741554)"
6,"Birchand Patel Path (Gardner Road), Patna, Pat...","(25.609587746739717, 85.12341035581215)"


In [735]:
import folium

In [736]:
mapa = folium.Map(location=[0, 0], zoom_start=2)

In [737]:
for l in range(len(df)):
    mapa.add_child(folium.Marker(location=df['Coordinates'][l],popup=folium.Popup(df['City Name'][l]),icon=folium.Icon(color='orange',icon='ok')))
    if l != len(df)-1:
        folium.PolyLine([df['Coordinates'][l], df['Coordinates'][l+1]], color='red',weight=2.5, opacity=0.8).add_to(mapa)
    else:
        folium.PolyLine([df['Coordinates'][l], df['Coordinates'][0]], color='red',weight=2.5, opacity=0.8).add_to(mapa)

In [740]:
mapa